In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# installing packages
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [ ]:
# loading dataset
from datasets import load_dataset

dataset = load_dataset("persiannlp/parsinlu_sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/13617 [00:00<?, ? examples/s]

Generating test_food split:   0%|          | 0/1344 [00:00<?, ? examples/s]

Generating test_movies split:   0%|          | 0/816 [00:00<?, ? examples/s]

Generating validation_food split:   0%|          | 0/1330 [00:00<?, ? examples/s]

Generating validation_movies split:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'review_id', 'example_id', 'excel_id', 'question', 'category', 'aspect', 'label', 'guid'],
        num_rows: 13617
    })
    test_food: Dataset({
        features: ['review', 'review_id', 'example_id', 'excel_id', 'question', 'category', 'aspect', 'label', 'guid'],
        num_rows: 1344
    })
    test_movies: Dataset({
        features: ['review', 'review_id', 'example_id', 'excel_id', 'question', 'category', 'aspect', 'label', 'guid'],
        num_rows: 816
    })
    validation_food: Dataset({
        features: ['review', 'review_id', 'example_id', 'excel_id', 'question', 'category', 'aspect', 'label', 'guid'],
        num_rows: 1330
    })
    validation_movies: Dataset({
        features: ['review', 'review_id', 'example_id', 'excel_id', 'question', 'category', 'aspect', 'label', 'guid'],
        num_rows: 360
    })
})

In [ ]:
dataset['train'][1]

{'review': 'دوستان حتما دقت کنید درقسمت فیله تکه های سبز رنگ داشت که نشان دهنده انتیبیوتیک هایی هست که به مرغ ها میدن و اون تکه سمیه. بیشترشم استخونه من راضی نبودم و دیگه ازین مارو خرید نمیکنم',
 'review_id': '1',
 'example_id': '2',
 'excel_id': 'food_1744',
 'question': 'نظر شما در مورد قیمت و ارزش خرید این گوشت مرغ چیست؟',
 'category': 'گوشت مرغ',
 'aspect': 'ارزش خرید',
 'label': '-3',
 'guid': 'food-train-r1-e2'}

In [ ]:
dataset['test_food'][1]

{'review': 'مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره\u200c و قیمتش مناسبه.',
 'review_id': '1726',
 'example_id': '2',
 'excel_id': 'food_197',
 'question': 'نظر شما در مورد قیمت و ارزش خرید این قهوه چیست؟',
 'category': 'قهوه',
 'aspect': 'ارزش خرید',
 'label': '1',
 'guid': 'food-test-r1726-e2'}

In [ ]:
dataset['test_movies'][1]

{'review': 'یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد میشود.',
 'review_id': '405',
 'example_id': '2',
 'excel_id': 'movie_152',
 'question': 'نظر شما در مورد داستان، فیلمنامه، دیالوگ ها و موضوع فیلم  درساژ چیست؟',
 'category': ' درساژ',
 'aspect': 'داستان',
 'label': '3',
 'guid': 'movie-test-r405-e2'}

In [ ]:
dataset['validation_movies'][1]

{'review': 'فیلمی که با تمام وجود روحتون رو آزار می ده. نمی دونم چرا موضوعات ناراحت کننده، نشون دادن بدبختی و زجر کشیدن آدمها، جدیدا اینقدر جذاب شده!!! دیدن این فیلم رو اصلا توصیه نمی کنم!',
 'review_id': '360',
 'example_id': '2',
 'excel_id': 'movie_338',
 'question': 'نظر شما در مورد داستان، فیلمنامه، دیالوگ ها و موضوع فیلم  همه چیز برای فروش چیست؟',
 'category': ' همه چیز برای فروش',
 'aspect': 'داستان',
 'label': '-1',
 'guid': 'movie-dev-r360-e2'}

In [ ]:
dataset['validation_food'][1]

{'review': 'من خیلی دنبال این طمع گشتم تا تو دیجی پیداش کردم . من از طعمش خوشم اومد البته ممکنه کسی هم خوشش نیاد',
 'review_id': '1536',
 'example_id': '2',
 'excel_id': 'food_425',
 'question': 'نظر شما در مورد قیمت و ارزش خرید این آدامس و خوشبو کننده\u200cی دهان چیست؟',
 'category': 'آدامس و خوشبو کننده\u200cی دهان',
 'aspect': 'ارزش خرید',
 'label': '-3',
 'guid': 'food-dev-r1536-e2'}

In [ ]:
   # this is dataset labels

    # '-3': 'no sentiment expressed',
    # '-2': 'very negative',
    # '-1': 'negative',
    # '0': 'neutral',
    # '1': 'positive',
    # '2': 'very positive',
    # '3': 'mixed',


In [ ]:
# our model need int label so wee need to map the label to int, since our model only has 3 classes we only need label -1,0,1 from dataset
label_mapping = {
    '-1': 0,  # negative
    '0': 1,   # neutral
    '1': 2    # positive
}

def filter_function(example):
    return example['label'] in label_mapping

# Filter dataset
filtered_train_dataset = dataset['train'].filter(filter_function)
filtered_validation_dataset = dataset['validation_movies'].filter(filter_function)

Filter:   0%|          | 0/13617 [00:00<?, ? examples/s]

Filter:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
set(filtered_train_dataset['label'])

{'-1', '0', '1'}

In [ ]:
from transformers import AutoTokenizer
# downloading model and tokenizer

model_name_or_path = "nimaafshar/parsbert-fa-sentiment-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# here I tokenize review which is a string text and I have int lables
def tokenize_function(examples):
    tokenized = tokenizer(examples['review'], padding="max_length", truncation=True, max_length=128)
    tokenized['labels'] = label_mapping[examples['label']]  # Convert labels to integers using the mapping
    return tokenized

tokenized_train_dataset = filtered_train_dataset.map(tokenize_function, batched=False, remove_columns=['review', 'review_id', 'example_id', 'excel_id', 'question', 'category', 'aspect', 'label', 'guid'])
tokenized_validation_dataset = filtered_validation_dataset.map(tokenize_function, batched=False, remove_columns=['review', 'review_id', 'example_id', 'excel_id', 'question', 'category', 'aspect', 'label', 'guid'])

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/2046 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

In [ ]:
# checking my data after tokenizing
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 2046
})

In [ ]:
tokenized_validation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 34
})

In [ ]:
tokenized_train_dataset[0]

{'input_ids': [2,
  5668,
  2799,
  2802,
  4735,
  3874,
  4642,
  5991,
  4752,
  2938,
  2786,
  2991,
  1348,
  21668,
  13238,
  12702,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
from peft import LoraConfig, TaskType
# LORA configuration object using Hugging Face’s Efficient Fine-Tuning (PEFT) parameters.
# The task_Type parameter specifies the task type for which the model will undergo fine-tuning.
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [ ]:
from transformers import BertForSequenceClassification
# Load a pre-trained BERT model for sequence classification with 3 labels
model = BertForSequenceClassification.from_pretrained(
    model_name_or_path,
    num_labels=3,
    from_tf = True
)

tf_model.h5:   0%|          | 0.00/652M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [ ]:
model

In [ ]:
from peft import get_peft_model
# Apply the PEFT (Parameter-Efficient Fine-Tuning) model with LoRA configuration to the pre-trained model
model = get_peft_model(model, lora_config)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
# Define a function to compute evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
# Set up training arguments, including output directory, evaluation strategy, and number of epochs
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/anewstart/karlancer/sentiment/test_trainer1", evaluation_strategy="epoch",
                                 num_train_epochs=25,)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
tokenized_train_dataset['labels'][0]

1

In [ ]:
set(tokenized_train_dataset['labels'])

{0, 1, 2}

In [ ]:
# # Enable detailed error reporting
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
!nvidia-smi

Mon Jul  1 08:20:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0              27W /  70W |   2165MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Print the labels to ensure they are correct
print("Labels in the tokenized train dataset:", [example['labels'] for example in tokenized_train_dataset])
print("Labels in the tokenized validation dataset:", [example['labels'] for example in tokenized_validation_dataset])

Labels in the tokenized train dataset: [1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 1, 0, 2, 2, 2, 2, 2, 1, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 

In [ ]:
# train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.964452,0.529412
2,1.192600,0.857226,0.558824
3,1.192600,0.836332,0.558824
4,0.708500,0.834161,0.588235
5,0.708500,0.816424,0.647059
6,0.691900,0.798127,0.705882
7,0.691900,0.773724,0.676471
8,0.659000,0.764156,0.676471
9,0.659000,0.742923,0.676471
10,0.604200,0.733032,0.735294


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=6400, training_loss=0.6235571074485778, metrics={'train_runtime': 816.1054, 'train_samples_per_second': 62.676, 'train_steps_per_second': 7.842, 'total_flos': 3366101591424000.0, 'train_loss': 0.6235571074485778, 'epoch': 25.0})

In [ ]:
# Save the trained model and tokenizer to the specified directory
model.save_pretrained("/content/drive/MyDrive/anewstart/karlancer/sentiment/results1")
tokenizer.save_pretrained("/content/drive/MyDrive/anewstart/karlancer/sentiment/results1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/anewstart/karlancer/sentiment/results1/tokenizer_config.json',
 '/content/drive/MyDrive/anewstart/karlancer/sentiment/results1/special_tokens_map.json',
 '/content/drive/MyDrive/anewstart/karlancer/sentiment/results1/vocab.txt',
 '/content/drive/MyDrive/anewstart/karlancer/sentiment/results1/added_tokens.json',
 '/content/drive/MyDrive/anewstart/karlancer/sentiment/results1/tokenizer.json')

# inference

In [ ]:
import torch
from peft import PeftModel
from transformers import  AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BertForSequenceClassification


In [ ]:
model_name_or_path = "nimaafshar/parsbert-fa-sentiment-twitter"

In [ ]:
# Load the entire model on the GPU 0
device_map = {"": 0}
# fine-tuned model path
path = "/content/drive/MyDrive/anewstart/karlancer/sentiment/results1"

# base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
base_model = BertForSequenceClassification.from_pretrained(model_name_or_path, from_tf=True)
# because I am using lora I need to use PeftModel to have my base model and fine-tuned model
model = PeftModel.from_pretrained(base_model, path)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [ ]:
model

In [ ]:
tokenizer

In [ ]:
# Example inputs
inputs = tokenizer("سلام", return_tensors='pt')

# Print inputs to verify their structure
print(inputs)

# Forward pass through the model
outputs = model(**inputs)

# Example of accessing logits or any other output
print(outputs)

{'input_ids': tensor([[   2, 4285,    4]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[ 0.7383, -1.1728,  0.1496]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [ ]:
print(outputs.logits)

tensor([[ 0.7383, -1.1728,  0.1496]], grad_fn=<AddmmBackward0>)


In [ ]:
classes = ["negative", "neutral", "positive"]
sequences = [
    ".غذا خیلی افتضاح بود متاسفم برای مدیریت رستورن خیلی بد بود.",
    "خیلی خوشمزده و عالی بود عالی",
    "می‌تونم اسمتونو بپرسم؟",
    "بی شخصیت"
]

# Iterate through sequences and make predictions
for sequence in sequences:
    # Tokenize input
    inputs = tokenizer(sequence, return_tensors="pt", max_length=128, padding="max_length", truncation=True)

    # Get logits from the model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Get predicted class
    predicted_class_idx = torch.argmax(probabilities, dim=1).item()
    predicted_class = classes[predicted_class_idx]

    # Convert probabilities to NumPy array for printing
    probabilities = probabilities.numpy()[0]

    print(f"Sequence: {sequence}")
    print(f"Predicted class: {predicted_class}")
    print(f"Probabilities: {probabilities}")
    print()

Sequence: .غذا خیلی افتضاح بود متاسفم برای مدیریت رستورن خیلی بد بود.
Predicted class: negative
Probabilities: [0.697853   0.16658133 0.13556565]

Sequence: خیلی خوشمزده و عالی بود عالی
Predicted class: positive
Probabilities: [0.00593248 0.01492997 0.97913754]

Sequence: می‌تونم اسمتونو بپرسم؟
Predicted class: negative
Probabilities: [0.80706173 0.05285115 0.14008713]

Sequence: بی شخصیت
Predicted class: negative
Probabilities: [0.8420616  0.02046818 0.13747028]



In [ ]:
classes = ["negative", "neutral", "positive"]
sequences = [
    "یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد",
    "من خیلی دنبال این طمع گشتم تا تو دیجی پیداش کردم . من از طعمش خوشم اومد البته ممکنه کسی هم خوشش نیاد",
    "دوستان حتما دقت کنید درقسمت فیله تکه های سبز رنگ داشت که نشان دهنده انتیبیوتیک هایی هست که به مرغ ها میدن و اون تکه سمیه. بیشترشم استخونه من راضی نبودم و دیگه ازین مارو خرید نمیکنم",

]

# Iterate through sequences and make predictions
for sequence in sequences:
    # Tokenize input
    inputs = tokenizer(sequence, return_tensors="pt", max_length=128, padding="max_length", truncation=True)

    # Get logits from the model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Get predicted class
    predicted_class_idx = torch.argmax(probabilities, dim=1).item()
    predicted_class = classes[predicted_class_idx]

    # Convert probabilities to NumPy array for printing
    probabilities = probabilities.numpy()[0]

    print(f"Sequence: {sequence}")
    print(f"Predicted class: {predicted_class}")
    print(f"Probabilities: {probabilities}")
    print()

Sequence: یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد
Predicted class: negative
Probabilities: [0.6923772  0.01759712 0.29002565]

Sequence: من خیلی دنبال این طمع گشتم تا تو دیجی پیداش کردم . من از طعمش خوشم اومد البته ممکنه کسی هم خوشش نیاد
Predicted class: negative
Probabilities: [0.5211526  0.02084494 0.45800242]

Sequence: دوستان حتما دقت کنید درقسمت فیله تکه های سبز رنگ داشت که نشان دهنده انتیبیوتیک هایی هست که به مرغ ها میدن و اون تکه سمیه. بیشترشم استخونه من راضی نبودم و دیگه ازین مارو خرید نمیکنم
Predicted class: negative
Probabilities: [0.92443126 0.02211499 0.05345375]

